In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_data(path):
    data=np.loadtxt(path)
    X=data[:, 1: ]
    y=data[:, 0]
    return X, y

X_train, y_train=get_data('./data/zip.train')
X_test, y_test=get_data('./data/zip.test')